![apr](Banner.jpg)

# Construcción de MDPs


Para el taller de MDPs, vamos a construir el caso sencillo de un del ambiente de los estados del motor de un vehículo, como se muestra en la siguiente figura.

![cars.png](cars_mdp.png)

El ambiente contiene tres estados en los que se puede encontrar un vehículo en movimiento, frio (`cold` en azul), caliente (`hot` en rojo), o recalentado (`overheated` en gris). El vehículo tiene dos acciones posibles, ir rápido (`fast`) o ir lento (`slow`), las cuales se muestran como una etiqueta roja sobre las transiciones entre los estados. Asociado a cada una de las transiciones, también encontramos la recompensa asociada de ejecutar la acción.

El trabajo de este taller consiste en construir un MDP que modele el ambiente mostrado en la figura

#### 1. Estructura del ambiente

Para definir el ambiente requerido utilizaremos una clase `MDP`. Esta clase recibe como parámetro una codificación de los estados, la dimensión del MDP dada por la cantidad de estados en el ambiente (dados como una tupla) y el estado inicial. 

La codificación del ambiente se puede realizar de varias formas. Por facilidad de uso más adelante en la ejecución de las funciones del ambiente, en este caso lo definiremos como una tabla de tamaño `estados x acciones` que para cada uno de los estados (filas) contiene una lista de tuplas con la información de la probabilidad asociada a la acción de la columna, el estado de llegada de ejecutar la acción y la recompensa asociada a la ejecución de la acción. La codificación de la tabla se muestra a continuación. Note que como una precondición, la definición de los estados debe ser correcta, enumerándolos como `cold=0`, `warm=1` y `overheated=2`.

![Encoding_table](table.png)

Las dimensiones del tablero se almacenan en los astributos `nrows` y `ncols` de la clase `MDP`.
Los valores codificados en el tablero se almacenarán en dos atributos de la clase, un atributo `transitions` que mantiene la probabilidad de transición de un estado a otro dada una acción como una lista de tuplas `(probability, state)` y un atributo `rewards` que mantiene la información las recompensas obtenida en el paso entre estados definido como una lista `(reward, state)`.

Adicionalmente la clase debe tener los atributos `initial_state` y `state` que corresponden al estado inicial, dado por parámetro, y el estado actual del agente en el ambiente, inicializado en el estado inicial.

Note también que el estado `overheated` no tienen ninguna acción asociada a él y por lo tanto lo consideramos como un estado final.

In [1]:
# Engine environment definition

# Required libraries
import random

# Definition of the main class
class MDP:
    def __init__(self, table: list[list[tuple]], dimensions: tuple[int, int], initial_state: int):
        self.model = table
        self.nrows, self.ncols = dimensions
        self.initial_state = initial_state
        self.state = initial_state
    
        self.action_map = {
            "slow": 0,
            "fast": 1
        }
        
        self.transitions = []
        self.rewards = []
        
        for state_actions in table:
            state_trans_list = []
            state_rews_list = []
            
            for action_results in state_actions:
                if action_results is None:
                    state_trans_list.append(None)
                    state_rews_list.append(None)
                else:
                    trans_tuples = []
                    rews_tuples = []
                    
                    for (prob, next_state, reward) in action_results:
                        trans_tuples.append((prob, next_state))
                        rews_tuples.append((reward, next_state))
                        
                    state_trans_list.append(trans_tuples)
                    state_rews_list.append(rews_tuples)
                    
            self.transitions.append(state_trans_list)
            self.rewards.append(state_rews_list)
        
    def get_current_state(self) -> int:
        return self.state
    
    def get_possible_actions(self, state: int) -> list[str]:
        possible_actions = []
        for action_name, action_idx in self.action_map.items():
            if self.model[state][action_idx] is not None:
                possible_actions.append(action_name)
        return possible_actions
        
    def do_action(self, action: str) -> tuple[float, int]:
        s = self.state
        
        if self.is_terminal():
            return 0.0, s
        
        action_idx = self.action_map.get(action)
        
        if action_idx is None:
            raise ValueError(f"Unknown action: {action}")
        
        results = self.model[s][action_idx]
        
        if results is None:
            return 0.0, s
        
        probabilities = [out[0] for out in results]
        next_states = [out[1] for out in results]
        rewards = [out[2] for out in results]
        
        chosen_idx = random.choices(range(len(results)), weights=probabilities, k=1)[0]
        
        next_s = next_states[chosen_idx]
        reward = rewards[chosen_idx]
        
        self.state = next_s
        
        return float(reward), next_s

    def reset(self):
        self.state = self.initial_state
    
    def is_terminal(self) -> bool:
        return len(self.get_possible_actions(self.state)) == 0


In [2]:
#Pruebas estructura del ambiente

table = [[[(1,0,1)], [(0.5, 0, 2), (0.5,1,2)]],
         [[(0.5, 0, 1), (0.5,1,1)], [(1, 2, -10)]],
         [None, None]]

env = MDP(table, (3,2), 0)

try:
    env.nrows
except:
    print("El atributo nrows no está definido")
try:
    env.ncols
except:
    print("El atributo ncols no está definido")
try:
    env.initial_state
except:
    print("El atributo initial_state no está definido")
try:
    env.state
except:
    print("El atributo state no está definido")


In [3]:
#Pruebas adicionales



#### 2. Acciones del agente

Defina la función `do_action` que ejecuta la acción tomada por el agente dentro del ambiente. Esta función recibe como parámetro la acción a ejecutar (el nombre de la acción a ejecutar dentro de las acciones posibles del agente) y retorna (una tupla con) el valor la recompensa del estado de llegada de la acción y el estado de llegada. 

Las acciones posibles del agente son `slow` y `fast` siguiendo la función de ruido dada por el MDP.


In [4]:
#Pruebas de ejecución de las acciones

table = [[[(1,0,1)], [(0.5, 0, 2), (0.5,1,2)]],
         [[(0.5, 0, 1), (0.5,1,1)], [(1, 2, -10)]],
         [None, None]]

env = MDP(table, (3,2), 0)

try:
    env.do_action
except:
    print("La función do_action no está definida")



In [5]:
#Pruebas adicionales



#### 3. Estado actual

Defina la función `get_current_state` que retorna el estado actual del agente. Esta función no recibe ningún parámetro y retorna el estado actual.

In [6]:
#Pruebas estado actual

table = [[[(1,0,1)], [(0.5, 0, 2), (0.5,1,2)]],
         [[(0.5, 0, 1), (0.5,1,1)], [(1, 2, -10)]],
         [None, None]]

env = MDP(table, (3,2), 0)

try:
    env.get_current_state
except:
    print("La función do_action no está definida")


In [7]:
#Pruebas adicionales



#### 4. Obtener las acciones

Defina la función `get_possible_actions` que recibe el estado actual del agente por parámetro y retorna una lista de las acciones válidas para el estado dado.

In [8]:
#Pruebas de acciones posibles

table = [[[(1,0,1)], [(0.5, 0, 2), (0.5,1,2)]],
         [[(0.5, 0, 1), (0.5,1,1)], [(1, 2, -10)]],
         [None, None]]

env = MDP(table, (3,2), 0)

try:
    env.get_possible_actions
except:
    print("La función get_possible_actions no está definida")
    


In [9]:
#Pruebas adicionales




#### 5. Reinicializar el ambiente

Defina la función `reset` que no recibe parámetros ni retorna ningún valor. El efecto de esta función es restablecer el ambiente a su estado inicial.



In [10]:
#Pruebas de reset

table = [[[(1,0,1)], [(0.5, 0, 2), (0.5,1,2)]],
         [[(0.5, 0, 1), (0.5,1,1)], [(1, 2, -10)]],
         [None, None]]

env = MDP(table, (3,2), 0)

try:
    env.reset
except:
    print("La función reset no está definida")


In [11]:
#Pruebas adicionales




#### 6. Estados terminales

Defina la función `is_terminal` que determina si el agente está en un estado final o no. En nuestro caso los estados finales o los estados de salida estarán determinados por las casillas con recompensa 1 o -1.
Esta función no recibe parámetros y retorna un booleano determinando si el agente está en un estado final o no.

In [12]:
#Pruebas terminación

table = [[[(1,0,1)], [(0.5, 0, 2), (0.5,1,2)]],
         [[(0.5, 0, 1), (0.5,1,1)], [(1, 2, -10)]],
         [None, None]]

env = MDP(table, (3,2), 0)

try:
    env.is_terminal
except:
    print("La función is_terminal no está definida")


#### Ejecución

Ahora vamos a ejecutar nuestro MDP.

Para la ejecución del MDP vamos a ejecutar las todas las acciones hasta encontrar el estado terminal, observando los estados por los que pasa el agente y la recompensa asociada acumulada de toda la ejecución.


In [16]:
import random

table = [[[(1,0,1)], [(0.5, 0, 2), (0.5,1,2)]],
         [[(0.5, 0, 1), (0.5,1,1)], [(1, 2, -10)]],
         [None, None]]

env = MDP(table, (3,2), 0)

done = False
total_reward = 0
steps = 0
while not done:
    old_state = env.get_current_state()
    action = random.choice(env.get_possible_actions(old_state))
    reward, new_state = env.do_action(action)
    total_reward += reward
    print(f"Ejecutando la acción {action} desde el estado {old_state} al estado {new_state}")
    steps += 1
    done = env.is_terminal()
print(f"Ejecuto {steps} pasos y obtuvo una recompensa de {total_reward}")

Ejecutando la acción fast desde el estado 0 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 0
Ejecutando la acción slow desde el estado 0 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 0
Ejecutando la acción slow desde el estado 0 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 1
Ejecutando la acción slow desde el estado 1 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 0
Ejecutando la acción slow desde el estado 0 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 0
Ejecutando la acción slow desde el estado 0 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 0
Ejecutando la acción fast desde el estado 0 al estado 1
Ejecutando la acción fast desde el estado 1 al e

## Preguntas de reflexión

¿Qué puede concluir de la ejecución del MDP en cuanto a la cantidad de acciones que se ejejcutan? ¿Existe alguna diferencia? 


## Reflexiones acerca de la salida conseguida:

1. El agent parte desde el estado inicial 0

2. Tiene dos acciones posibles (0,1) con transiciones de probabilidad ya definidas

3. Las transicoines con probabilidad < 1 hacen que varias veces el agent repita el mismo estado y eso se representqa en varias líneas repetias como por ejemplo:

`Ejecutando la acción 0 desde el estado 0 al estado 0`

`Ejecutando la acción 0 desde el estado 0 al estado 0`

`Ejecutando la acción 0 desde el estado 0 al estado 0`

`Ejecutando la acción 0 desde el estado 0 al estado 0`

`Ejecutando la acción 0 desde el estado 0 al estado 0`

`Ejecutando la acción 0 desde el estado 0 al estado 0`


4. Tras varios intentos el agent finalmente logra llegar al estado 2 que es terminal y acumula una reward total de 26.